In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import math
import pandas as pd
import numpy as np
import pickle
import seaborn as sbn
from pymongo import MongoClient
import matplotlib.pyplot as plt

sys.path.append("src/main/python")

In [7]:
import trainer
import train_tfidf
import train_pmodel
import train_lmodel

In [8]:
# Load our dataset, split the dataset into train and test
# Then save it off into training and test sets

dataset_path = "dataset.csv"
train_path = "train.csv"
test_path = "test.csv"
force_refresh = False

# Roughly 80/20 split
train_size = 300000

df = trainer.load(dataset_path, force_refresh)
shuf = np.random.random_integers(df.shape[0], size=df.shape[0])
print df.shape

train = df.iloc[shuf[:train_size]]
print train.shape
train.to_csv(train_path, encoding='utf-8')

test = df.iloc[shuf[train_size:]]
test = test.loc[pd.notnull(test['airport'])]
print test.shape
test.to_csv(test_path, encoding='utf-8')

[trainer] loading existing data at dataset.csv
(398120, 24)
(300000, 24)
(7426, 24)


In [ ]:
# Load the dataset
import trainer
import train_tfidf
import train_img
import train_pmodel
import train_lmodel

cache_collection = 'object_cache'

host = 'alfred.local'
port = 27017
db = 'instagram'
client = MongoClient(host, port)
cache = client[db][cache_collection]

# Define the extractor function we want to use
# and the recommendation model that we want to train
ex = train_tfidf.user_img
rm = train_lmodel.post_knn
model_template_path = "models/train_knnmodel_%d.pickle"

# Put your configs you want to try here
cfgs = [
    { 'k': 5, 'weights': 'uniform' },
    { 'k': 5, 'weights': 'distance' },
    { 'k': 7, 'weights': 'uniform' },
    { 'k': 7, 'weights': 'distance' },
    { 'k': 9, 'weights': 'uniform' },
    { 'k': 9, 'weights': 'distance' },
    { 'k': 11, 'weights': 'uniform' },
    { 'k': 11, 'weights': 'distance' },
    { 'k': 13, 'weights': 'uniform' },
    { 'k': 13, 'weights': 'distance' },
    { 'k': 15, 'weights': 'uniform' },
    { 'k': 15, 'weights': 'distance' },
    { 'k': 17, 'weights': 'uniform' },
    { 'k': 17, 'weights': 'distance' },
    { 'k': 19, 'weights': 'uniform' },
    { 'k': 19, 'weights': 'distance' },
    { 'k': 21, 'weights': 'uniform' },
    { 'k': 21, 'weights': 'distance' },
    { 'k': 23, 'weights': 'uniform' },
    { 'k': 23, 'weights': 'distance' },
    { 'k': 25, 'weights': 'uniform' },
    { 'k': 25, 'weights': 'distance' },
]
res = { 'cfg': [], 'precision': [], 'recall': [], 'f1': [] }

# For each config, train a model
for i,cfg in enumerate(cfgs):
    print >> sys.stderr, "Training %s" % cfg
    tr = trainer.Trainer(ex, rm, cache)
    m, metrics = tr.run(model_template_path % i, train_path, df_test = test, **cfg)
    res['cfg'].append(cfg)
    res['precision'].append(metrics['precision'])
    res['recall'].append(metrics['recall'])
    res['f1'].append(metrics['f1'])
    
res = pd.DataFrame(res)   
print res

plt.figure(figsize=(20, 10))
plt.subplot(1, 3, 1)
plt.bar(range(res.shape[0]), res['precision'])
plt.title('Precision')
plt.xlabel('Job #')
plt.xticks(range(res.shape[0]))

plt.subplot(1, 3, 2)
plt.bar(range(res.shape[0]), res['recall'])
plt.title('Recall')
plt.xlabel('Job #')
plt.xticks(range(res.shape[0]))

plt.subplot(1, 3, 3)
plt.bar(range(res.shape[0]), res['f1'])
plt.title('F1')
plt.xlabel('Job #')
plt.xticks(range(res.shape[0]))

plt.show()

[trainer] loading existing data at train.csv
Post BOW Dataset: (22252, 126079), (22252,)